 # 과제#1. car seat 매출 예측

 ![](https://cdn.images.express.co.uk/img/dynamic/24/590x/child-car-seat-986556.jpg?r=1532946857754)

 * 비즈니스 상황
     * 고객사는 국내(미국)와 국외에서 카시트를 판매하는 회사입니다.
     * 최근 경쟁사의 공격적인 마케팅으로 매출이 감소하고 있습니다.
     * 이를 해결하고자 여러분들에게 예측 모델링을 의뢰하였습니다.



 |	변수명	|	설명	|	구분	|
 |	----	|	----	|	----	|
 |	Sales 	|	 각 지역 판매액(단위 : 1000달러)	|	Target	|
 |	CompPrice 	|	지역별 경쟁사 판매가격(달러)	|	feature	|
 |	Advertising 	|	 각 지역, 회사의 광고 예산(단위 : 1000달러)	|	feature	|
 |	Population 	|	 지역 인구수(단위 : 1000명)	|	feature	|
 |	Price 	|	 자사 지역별 판매가격(달러)	|	feature	|
 |	ShelveLoc 	|	 진열상태	|	feature	|
 |	Age 	|	 지역 인구의 평균 연령	|	feature	|
 |	US 	|	 매장이 미국에 있는지 여부	|	feature	|
 |	Urban 	|	 매장이 도시에 있는지 여부	|	feature	|
 |	Education 	|	 평균학력수준(범주 : 11~17)	|	feature	|


 * 데이터
     * data : 학습과 검증용
     * test : **팀과제용** 입니다.
         * feature만 있습니다.
         * 운영에서 발생된 데이터로 간주하고, 예측한 후에 Kaggle에 업로드합니다.

 * 주의사항
     * data 에 대해 수행한 전처리 코드를 **순서에 맞춰** test에도 적용해야 함.

 ## 0.환경준비

 ### 0.1 Import

In [1]:
# 기본 라이브러리 가져오기
from distutils.log import warn
from random import Random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import (
    LinearRegression,
    LogisticRegression,
    LogisticRegressionCV,
)
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor

 ### 0.2 Data Loading

In [2]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/Carseat_train.csv"
data = pd.read_csv(path)
data.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,3.72,139.0,111.0,5,310,132,Bad,62,13,Yes,Yes
1,2.93,143.0,21.0,5,81,160,Medium,67,12,No,Yes
2,4.53,114.0,NaN,0,97,125,Medium,29,12,Yes,No
3,8.47,119.0,88.0,10,170,101,Medium,61,13,Yes,Yes
4,11.85,136.0,81.0,15,425,120,Good,67,10,Yes,Yes


 * 아래 데이터는 **팀 과제**를 위해 필요합니다.

In [3]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/Carseat_test_x.csv"

test = pd.read_csv(path)
test.head()

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,131.0,111.0,13,33,80,Bad,68,18,Yes,Yes
1,123.0,57.0,0,66,105,Medium,39,11,Yes,No
2,136.0,60.0,7,303,147,Medium,41,10,Yes,Yes
3,143.0,77.0,25,448,156,Medium,43,17,Yes,Yes
4,111.0,75.0,1,377,108,Good,25,12,Yes,No


 ## 1.데이터 탐색

 ## 2.데이터 전처리

In [4]:
target = "Sales"

 ### 2.1 데이터를 feature와 , target으로 분할

In [5]:
data.dropna(axis=0, inplace=True)

 ### 2.2 NaN에 대한 조치
 * 데이터에 포함된 NaN에 대해서 조치 방법을 결정하고 적용합니다.
 * 조치방법을 결정한 이유를 주석으로 기술합니다.
 상단에 원본데이터에서 처리

 ### 2.3 가변수화
 * 범주형 데이터이면서 값이 0,1 로 되어 있는 것이 아니라면, 가변수화를 수행해야 합니다.
 * 대상이 되는 변수에 대해서 가변수화를 수행해주세요.

In [6]:
e = ["ShelveLoc", "Urban", "US"]
for col in e:
    one_hot = pd.get_dummies(data[col], prefix=col)
    data = data.drop(col, axis=1)
    data = data.join(one_hot)

In [7]:
for col in ["Sales"]:
    data[col] = data[col].astype("int")
for col in [
    "Urban_No",
    "Urban_Yes",
    "US_No",
    "US_Yes",
]:
    data[col] = data[col].astype("category")
x = data.drop(target, axis=1)
y = data.loc[:, target]
data.dtypes

Sales                  int64
CompPrice            float64
Income               float64
Advertising            int64
Population             int64
Price                  int64
Age                    int64
Education              int64
ShelveLoc_Bad          uint8
ShelveLoc_Good         uint8
ShelveLoc_Medium       uint8
Urban_No            category
Urban_Yes           category
US_No               category
US_Yes              category
dtype: object

In [8]:
y

0       3
1       2
3       8
4      11
6       7
       ..
295     7
296     4
297     9
298     5
299    10
Name: Sales, Length: 275, dtype: int64

 ### 2.5 데이터 분할
 * train 데이터를 train : val 로 분할하시오.
 * train set의 적절한 사이즈를 결정하시오.

In [9]:
x, X, y, Y = train_test_split(x, y, test_size=0.3, random_state=2022)

 ### 2.6 (옵션)스케일링
 * 스케일링을 필요로 하는 알고리즘이 있습니다.
 * 필요하다고 판단될때 수행합니다.

 ## 3.모델링1 : 머신러닝

 ### 3.1 모델링을 위한 함수 불러오기

 * 꼭 필요한 함수만 불러옵니다. 무조건 다 불러오는 것은 지양해주세요.

 ### 3.2 모델링
 * 최소 3개 이상의 알고리즘을 선정합니다.(알고리즘에는 최소 1개 이상의 앙상블 기법을 포함)
     * 아래 주어진 목록 이외의 알고리즘을 선정해도 좋습니다.
 * 각 알고리즘별 튜닝을 수행하여 최적의 모델을 생성합니다.
     * 튜닝 방식은 GridSearchCV, RandomizedSearchCV 혹은 for loop 로 직접
     * 단, 선형회귀, 로지스틱 회귀의는 입력 변수(feature)를 조절하여 튜닝을 합니다.
 * 모델링, 튜닝만 수행하는데 그치지 말고, 모델의 내용을 파악하기 위한 분석을 시도합니다.
     * 예 : 모델 시각화 및 내용 파악, 튜닝 결과 분석 및 시각화

 %% [markdown]

In [10]:
import warnings

warnings.filterwarnings("ignore")

params = {
    # "min_samples_split": [2, 3],
    "fit_intercept": [True, False],
    "normalize": [True, False],
    "copy_X": [True, False],
}
dists = {
    "randomforestclassifier__max_depth": [3, 5, 10, 15],
    "randomforestclassifier__max_features": [3, 5, 10],
    "randomforestclassifier__n_estimators": [80, 100, 150, 200],
}

classifiers = [
    GridSearchCV(
        LinearRegression(),
        param_grid={
            "fit_intercept": [True, False],
            "normalize": [True, False],
            "copy_X": [True, False],
        },
        cv=10,
        refit=True,
    ),  # 1
    GridSearchCV(
        DecisionTreeRegressor(),
        param_grid={
            "max_depth": [1, 3, 5],
            "max_leaf_nodes": [2, 4, 6, 8],
        },
        cv=10,
        refit=True,
    ),  # 2
    GridSearchCV(
        KNeighborsRegressor(),
        param_grid={
            "leaf_size": [i for i in range(5, 100, 5)],
        },
        cv=10,
        refit=True,
    ),  # 3
    GridSearchCV(
        SVC(),
        param_grid={"max_iter": [i for i in range(5, 100, 5)]},
        cv=10,
        refit=True,
    ),  # 4
    GridSearchCV(
        RandomForestRegressor(),
        param_grid={"n_estimators": [i for i in range(50, 1000, 50)]},
        cv=10,
        refit=True,
    ),  # 5
]


for model in classifiers:
    model.fit(x, y)
    pred = model.predict(X)
    class_name = model.estimator.__class__.__name__
    print("{0:50} 정확도: {1:.4f}".format(class_name, model.score(X, Y)))

LinearRegression                                   정확도: 0.8627
DecisionTreeRegressor                              정확도: 0.3893
KNeighborsRegressor                                정확도: 0.1140
SVC                                                정확도: 0.0723
RandomForestRegressor                              정확도: 0.6626


 ## 4.모델링2 : 딥러닝

 ### 4.1 모델링을 위한 함수 불러오기

 ### 4.2 모델 설계
 * 다양한 구조의 모델 3개 이상을 설계하시오. (히든레이어, 노드 수 조절)

 * 모델1

 * 모델2

 * 모델3

 ### 4.3 학습 및 최적화
 * 다양한 학습조건으로 모델링 수행후 성능을 비교해 봅시다.


 ## 5.성능 비교

 * 알고리즘별 선정된 모델이 최소 6개 입니다. 이에 대해 성능을 평가합니다.
 * 평가를 위한 적절한 지표(metric)을 결정하고 비교합니다.
 * 최종 모델을 선정하게 된 근거를 주석으로 기술하시오.

 ## 6.[팀과제]Test set으로 예측
 * 전처리 코드 실행
 * 예측
 * 성능 튜닝 : Kaggle에서의 평가는 MAE로 하게 됩니다.

 #### 6.1 전처리 코드실행

 * 주의!!!
     * 전처리 코드는 2번에서의 순서를 반드시 지켜서 실행해야 합니다.
     * 중간에 오류가 나면, 순서가 맞지 않기때문에 발생될 수 있습니다.


In [11]:
# 가변수화 코드 실행

In [12]:
# NAN 채우기

In [13]:
# 경쟁사와 가격 차 변수 만들기

In [14]:
# 스케일링

 #### 6.2 예측

 * 5에서 가장 좋은 성능의 모델로 예측을 시도합니다.
 * 결과를 csv 파일로 저장해서 업로드 합니다.